<a href="https://colab.research.google.com/github/gseetha04/LLMs-Medicaldata/blob/main/Transfer_learning_GD_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate seqeval datasets>=1.8.0 torch>=1.3

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# tokenizer
from transformers import AutoTokenizer, DistilBertTokenizerFast
# sequence tagging model + training-related
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import torch
import json
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
for i, x in enumerate(y_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw[i][j] = a.replace('CT', 'O')

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'C', 1: 'O', 2: 'E'}


In [ ]:
X_train_final = np.array(X_raw)
y_train_final = np.array(y_raw)

<ipython-input-9-aff0ef744a5d>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_raw)
<ipython-input-9-aff0ef744a5d>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_raw)


In [ ]:
import os
k=2
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
metrics=[]
i=0
for train_index, val_index in kf.split(X_train_final,y_train_final):
  train_texts, dev_texts =X_train_final[train_index].tolist(),X_train_final[val_index].tolist()
  train_labels, dev_labels = y_train_final[train_index].tolist(),y_train_final[val_index].tolist()

In [ ]:
######test data############

In [ ]:
data_test = pd.read_excel('/content/data_GD_final.xlsx',sheet_name='Sheet2')

In [ ]:
import pandas as pd
import spacy
import re
import json
from nltk.tokenize import word_tokenize

data_train_df = pd.read_excel('/content/data_GD_final.xlsx',sheet_name='Sheet2')
tagged_words = []
for para in data_train_df['final']:
   para=para.encode("ascii", "ignore")
   para=para.decode()
   connl_text= []
   label = []
   cause = re.findall(r'<C>(.*?)</C>', para)
   #condition = re.findall(r'<CO>(.*?)</CO>', para)
   effect = re.findall(r'<E>(.*?)</E>', para)
   #action = re.findall(r'<A>(.*?)</A>', para)
   #signal = re.findall(r'<S>(.*?)</S>',para)
  # ARG0 = [re.sub('<(\w+)>(.*?)<\/\w+>','',i) for i in ARG0]
   splitted = re.split('<C>|</C>|<E>|</E>', para)  # splitted considering tags
   if cause or effect:  # if any tag in sentence
    for split in splitted:  # search each index
            if split in cause:
                for token in split.split():
                    connl_text.append(token)
                    label.append('C')
            elif split in effect:
                for token in split.split():
                    connl_text.append(token)
                    label.append('E')
            # elif split in signal:
            #     for token in split.split():
            #       connl_text.append(token)
            #       label.append('S')
            # elif split in action:
            #     for token in split.split():
            #       connl_text.append(token)
            #       label.append('A')
            # elif split in condition:
            #     for token in split.split():
            #       connl_text.append(token)
            #       label.append('CO')

            else:  # current word is not an entity
                for token in split.split():
                    connl_text.append(token)
                    label.append('O')

    tagged_words.append({'Word': connl_text, 'Tag': label})

In [ ]:
data_test = pd.DataFrame(tagged_words)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data_test[["Word", "Tag"]].rename(columns={"Word": "X", "Tag": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

In [ ]:
X_text_new = pd.DataFrame(X_raw,columns=['sentences'])
y_text_new = pd.DataFrame(y_raw,columns=['label'])

In [ ]:
df_test_new = pd.concat([X_text_new,y_text_new],axis=1)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_new = df_test_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

In [ ]:
import numpy as np
print(np.shape(X_raw_test))

(177,)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
X_raw_test

[['Pregnant',
  'persons',
  'with',
  'gestational',
  'diabetes',
  'are',
  'at',
  'increased',
  'risk',
  'for',
  'maternal',
  'and',
  'fetal',
  'complications,',
  'including',
  'preeclampsia,',
  'fetal',
  'macrosomia',
  '(which',
  'can',
  'cause',
  'shoulder',
  'dystocia',
  'and',
  'birth',
  'injury),',
  'and',
  'neonatal',
  'hypoglycemia',
  '.'],
 ['Gestational',
  'diabetes',
  'has',
  'also',
  'been',
  'associated',
  'with',
  'an',
  'increased',
  'risk',
  'of',
  'several',
  'long-term',
  'health',
  'outcomes',
  'in',
  'pregnant',
  'persons',
  'and',
  'intermediate',
  'outcomes',
  'in',
  'their',
  'offspring',
  '.'],
 ['EVIDENCE',
  'ASSESSMENT',
  'The',
  'USPSTF',
  'concludes',
  'with',
  'moderate',
  'certainty',
  'that',
  'there',
  'is',
  'a',
  'moderate',
  'net',
  'benefit',
  'to',
  'screening',
  'for',
  'gestational',
  'diabetes',
  'at',
  '24',
  'weeks',
  'of',
  'gestation',
  'or',
  'after',
  'to',
  'impr

In [ ]:
# for i, x in enumerate(y_raw_test):
#     for j, a in enumerate(x):
#         if 'S' in a:
#             y_raw_test[i][j] = a.replace('S', 'O')
#         if 'A' in a:
#             y_raw_test[i][j] = a.replace('A', 'O')
#         if 'CO' in a:
#             y_raw_test[i][j] = a.replace('CO', 'O')

In [ ]:
unique_tags = set(label for labels in y_raw_test for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'C', 1: 'O', 2: 'E'}


In [ ]:
RANDOM_SEED = 42
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_raw_test, y_raw_test, test_size=.1, random_state=RANDOM_SEED)

In [ ]:
print(np.shape(X_train_raw))
print(np.shape(X_test_raw))

(159,)
(18,)


In [ ]:
# X_test_raw = np.array(X_test_raw)
# y_test_raw = np.array(y_test_raw)

In [ ]:
X_train_final = train_texts + X_test_raw
y_train_final = train_labels + y_test_raw

In [ ]:
print(np.shape(train_texts))
print(np.shape(X_train_final))
print(np.shape(train_labels))
print(np.shape(y_train_final))

(1117,)
(1135,)
(1117,)
(1135,)


In [ ]:
print(np.shape(train_texts))
print(np.shape(X_train_final))
print(np.shape(train_labels))
print(np.shape(y_train_final))

(1117,)
(1135,)
(1117,)
(1135,)


In [ ]:
#X_train_final = np.array(X_train_final)

In [ ]:
X_train_final

[['When ',
  'a ',
  'policyholder ',
  'or ',
  'insured ',
  'gets ',
  'sick ',
  'or ',
  'hurt ',
  'the ',
  'Company ',
  'pays ',
  'cash ',
  'benefits ',
  'fairly ',
  'and ',
  'promptly ',
  'for ',
  'eligible ',
  'claims'],
 ['During ',
  '2020 ',
  'in ',
  'response ',
  'to ',
  'the ',
  'onset ',
  'and ',
  'development ',
  'of ',
  'the ',
  'global ',
  'COVID ',
  '19 ',
  'pandemic ',
  'the ',
  'Company ',
  'pivoted ',
  'to ',
  'digital ',
  'sales ',
  'methods ',
  'and ',
  'accelerated ',
  'related ',
  'digital ',
  'investments'],
 ['Conversely ',
  'a ',
  'rise ',
  'in ',
  'interest ',
  'rates ',
  'could ',
  'improve ',
  'the ',
  'Company ',
  's ',
  'ability ',
  'to ',
  'earn ',
  'higher ',
  'rates ',
  'of ',
  'return ',
  'on ',
  'future ',
  'investments ',
  'as ',
  'well ',
  'as ',
  'floating ',
  'rate ',
  'investments ',
  'held ',
  'in ',
  'its ',
  'investment ',
  'portfolio ',
  'However ',
  'an ',
  'increase ',

In [ ]:
model_name = "distilbert-base-uncased" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_final, X_train_final]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [dev_texts, dev_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# # ensure our data is well-formed:
# for ds in [X_train_raw, X_train_raw]:
#   for row in ds:
#     assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        # print(arr_offset)
        # print(type(arr_offset))

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())
        #print(encoded_labels)
    return encoded_labels

In [ ]:
#zero-shot
# encoded X
X_train = tokenize(train_texts)
X_dev = tokenize(dev_texts)
X_test= tokenize(X_raw_test)

y_train = encode_tags(tags=train_labels, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=dev_labels, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
# encoded X - few-shot
X_train = tokenize(X_train_final)
X_dev = tokenize(dev_texts)
X_test= tokenize(X_train_raw)

y_train = encode_tags(tags=y_train_final, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=dev_labels, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=y_train_raw, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.005500,0.973066
2,0.636000,0.580692
3,0.506600,0.496743


In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=4,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.034800,0.996188
2,0.807300,0.699045
3,0.417100,0.510397
4,0.426400,0.470714


In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#zero-shot
import wandb

run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=100,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
run.finish()

wandb: Currently logged in as: sgopala4. Use `wandb login --relogin` to force relogin


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.016200,0.983348
2,0.685400,0.634542
3,0.445800,0.494788
4,0.330500,0.474077
5,0.277600,0.511433
6,0.255000,0.527652
7,0.287500,0.511805
8,0.201900,0.597231
9,0.140500,0.660307
10,0.108100,0.690173


KeyboardInterrupt: ignored

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=15,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.007800,0.978320
2,0.675000,0.594174
3,0.513900,0.490751
4,0.360800,0.472983
5,0.335000,0.512649
6,0.223500,0.538206
7,0.179200,0.645943
8,0.156600,0.623534
9,0.120000,0.720923
10,0.093200,0.666743


In [ ]:
# pred = trainer.predict(test_dataset)
# preds = pred.predictions.argmax(-1)

In [ ]:
preds

array([[1, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 2],
       ...,
       [1, 1, 1, ..., 1, 2, 2],
       [1, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 1]])

In [ ]:
preds.shape

(159, 424)

In [ ]:
np.shape(y_test)

(159, 424)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in X_raw_test for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(67416,)
(67416,)
      index        texts
0         0  <S>Pregnant
1         1  persons</S>
2         2         with
3         3  gestational
4         4     diabetes
...     ...          ...
6462     29           in
6463     30          the
6464     31      absence
6465     32           of
6466     33   screening.

[6467 rows x 2 columns]
      index        texts
0         0  <S>Pregnant
1         1  persons</S>
2         2         with
3         3  gestational
4         4     diabetes
...     ...          ...
6462     29           in
6463     30          the
6464     31      absence
6465     32           of
6466     33   screening.

[6467 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'C', 1: 'O', 2: 'E'}


In [ ]:
targetname = ['C', 'O', 'E']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)
#no determiners


       preds  gold_data
0          1       -100
1          1          1
2          1          1
3          1          1
4          1       -100
...      ...        ...
67411      2       -100
67412      2       -100
67413      2       -100
67414      2       -100
67415      1       -100

[67416 rows x 2 columns]
       preds  gold_data
1          1          1
2          1          1
3          1          1
7          1          1
12         1          1
...      ...        ...
67015      2          2
67016      2          2
67017      2          2
67018      2          2
67019      1          1

[5840 rows x 2 columns]


In [ ]:
#distilbert base uncased - 3 epochs second run
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.45      0.48      0.47      1812
           C       0.37      0.08      0.13      1072
           O       0.75      0.84      0.79      6489

    accuracy                           0.68      9373
   macro avg       0.53      0.46      0.46      9373
weighted avg       0.65      0.68      0.65      9373



In [ ]:
#distilbert base uncased - 4 epochs zer-shot
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.29      0.67      0.40      2009
           O       0.83      0.26      0.40      7133
           C       0.19      0.54      0.28      1159

    accuracy                           0.37     10301
   macro avg       0.43      0.49      0.36     10301
weighted avg       0.65      0.37      0.39     10301



In [ ]:
#distilbert base uncased - 4 epochs zer-shot only cause/effect labels
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.40      0.72      0.51      2009
           O       0.89      0.20      0.32      3299
           C       0.31      0.56      0.40      1159

    accuracy                           0.43      6467
   macro avg       0.53      0.49      0.41      6467
weighted avg       0.63      0.43      0.40      6467



In [ ]:
#distilbert base uncased - 4 epochs zer-shot only cause/effect labels - final
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.74      0.43      0.54      3299
           E       0.44      0.68      0.53      2009
           C       0.35      0.43      0.38      1159

    accuracy                           0.51      6467
   macro avg       0.51      0.51      0.49      6467
weighted avg       0.57      0.51      0.51      6467



In [ ]:
#distilbert base uncased - 4 epochs few-shot
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.81      0.80      0.80      6489
           C       0.28      0.28      0.28      1072
           E       0.58      0.60      0.59      1812

    accuracy                           0.70      9373
   macro avg       0.55      0.56      0.56      9373
weighted avg       0.70      0.70      0.70      9373



In [ ]:
#distilbert base uncased - 4 epochs few-shot only cause/effect labels
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.77      0.50      0.60      3044
           E       0.50      0.77      0.61      1795
           C       0.43      0.46      0.44      1001

    accuracy                           0.58      5840
   macro avg       0.56      0.58      0.55      5840
weighted avg       0.63      0.58      0.58      5840



In [ ]:
#distilbert base uncased - 4 epochs few-shot only cause/effect labels - final
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.44      0.49      0.46      1001
           O       0.76      0.64      0.69      3044
           E       0.57      0.69      0.62      1795

    accuracy                           0.63      5840
   macro avg       0.59      0.60      0.59      5840
weighted avg       0.65      0.63      0.63      5840



In [ ]:
#distilbert base uncased - 3 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.17      0.68      0.27      1159
           E       0.29      0.59      0.39      2009
           O       0.90      0.18      0.30      7133

    accuracy                           0.32     10301
   macro avg       0.45      0.48      0.32     10301
weighted avg       0.70      0.32      0.31     10301



In [ ]:
#distilbert base uncased - 15 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.18      0.49      0.26      1159
           E       0.34      0.72      0.46      2009
           O       0.82      0.33      0.48      7133

    accuracy                           0.43     10301
   macro avg       0.45      0.51      0.40     10301
weighted avg       0.66      0.43      0.45     10301



In [ ]:
#################BioBERT############################

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
model_name = "dmis-lab/biobert-base-cased-v1.2" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
import wandb

run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=100,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
run.finish()

eval/loss,█▄▁▇▅
eval/runtime,▂█▁▆▁
eval/samples_per_second,▇▁█▃█
eval/steps_per_second,▇▁█▃█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/loss,█▇▇▆▅▅▅▄▄▄▄▅▄▄▃▃▃▂▄▃▃▃▂▂▂▃▂▂▂▁▂▂▁▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['transformer.layer.1.ffn.lin2.weight', 'transformer.layer.11.sa_layer_norm.weight', 'transformer.layer.3.attention.q_lin.weight', 'transformer.layer.9.output_layer_norm.weight', 'transformer.layer.1.sa_layer_norm.bias', 'transformer.layer.3.sa_layer_norm.weight', 'transformer.layer.3.attention.out_lin.bias', 'transformer.layer.8.ffn.lin2.bias', 'transformer.layer.1.ffn.lin1.bias', 'transformer.layer.6.attention.k_lin.bias', 'transformer.layer.6.attention.v_lin.bias', 'transformer.layer.2.attention.out_lin.bias', 'transformer.layer.3.attention.out_lin.weight', 'transformer.layer.3.ffn.lin1.bias', 'transformer.layer.5.ffn.lin1.weight', 'embeddings.position_embeddings.weight', 't

Epoch,Training Loss,Validation Loss
1,1.048100,1.014721
2,1.003000,0.969995
3,0.967300,0.976295
4,0.927600,1.014704
5,0.883800,0.987501
6,0.878300,1.059924
7,0.804300,0.955797
8,0.680800,0.919056
9,0.611900,0.991270
10,0.433300,1.109033


KeyboardInterrupt: ignored

In [ ]:
import wandb

run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=100,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
run.finish()

eval/loss,█▄▁▁
eval/runtime,█▂▁▂
eval/samples_per_second,▁▇█▇
eval/steps_per_second,▁▇█▇
train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/learning_rate,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
train/loss,███▇▇▇▇▆▆▆▆▆▅▅▄▃▃▂▂▂▂▁▁▁▂▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['transformer.layer.6.attention.q_lin.bias', 'transformer.layer.7.ffn.lin1.bias', 'transformer.layer.1.ffn.lin1.weight', 'transformer.layer.2.ffn.lin1.weight', 'transformer.layer.5.attention.out_lin.bias', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.1.ffn.lin2.bias', 'transformer.layer.8.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.10.sa_layer_norm.weight', 'transformer.layer.11.ffn.lin1.bias', 'transformer.layer.4.attention.k_lin.weight', 'transformer.layer.2.attention.k_lin.bias', 'transformer.layer.11.output_layer_norm.bias', 'transformer.layer.3.attention.out_lin.weight', 'transformer.layer.8.ffn.lin2.weight', 'transformer.layer.4.ffn.lin2.weight', 'transformer.layer.3.output_layer_norm.bias', 'transformer.layer.2.output_layer_n

Epoch,Training Loss,Validation Loss
1,1.040800,1.007993
2,0.991300,0.971558
3,0.960300,0.948989
4,0.924400,0.965499
5,0.889900,0.956978
6,0.869100,0.953002
7,0.766500,0.941784
8,0.661000,0.928346
9,0.600000,1.002044
10,0.501800,1.012141


eval/loss,▁▁▁▁▂▂▃▃▄▄▄▄▅▆▆▆▆▇▆▆▆▇▇▇▇█▇█▇█▇███▇█████
eval/runtime,▂▃▄▄▃▄▃▃▃▃▃▃▄▄▃▄▃▂▃▃▃▂▃▃▃▄▃▃█▄▃▇▂▄▁▃▃▃▃▂
eval/samples_per_second,▇▆▅▅▅▅▆▆▆▆▆▆▅▅▆▅▆▆▆▆▆▇▆▆▆▅▅▆▁▅▆▂▇▅█▆▆▆▆▇
eval/steps_per_second,▇▆▅▅▆▅▆▆▆▆▆▆▅▅▆▅▆▆▆▆▆▇▆▆▆▅▆▆▁▅▆▂▇▅█▆▆▆▆▇
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▂▄▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▇▇▆▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
# import wandb

# run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=5,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# run.finish()

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['transformer.layer.1.ffn.lin2.weight', 'transformer.layer.11.sa_layer_norm.weight', 'transformer.layer.3.attention.q_lin.weight', 'transformer.layer.9.output_layer_norm.weight', 'transformer.layer.1.sa_layer_norm.bias', 'transformer.layer.3.sa_layer_norm.weight', 'transformer.layer.3.attention.out_lin.bias', 'transformer.layer.8.ffn.lin2.bias', 'transformer.layer.1.ffn.lin1.bias', 'transformer.layer.6.attention.k_lin.bias', 'transformer.layer.6.attention.v_lin.bias', 'transformer.layer.2.attention.out_lin.bias', 'transformer.layer.3.attention.out_lin.weight', 'transformer.layer.3.ffn.lin1.bias', 'transformer.layer.5.ffn.lin1.weight', 'embeddings.position_embeddings.weight', 't

Epoch,Training Loss,Validation Loss
1,1.042800,1.011827
2,0.996700,0.973675
3,0.956300,0.951948
4,0.924500,1.000995
5,0.879200,0.987246


In [ ]:
# import wandb

# run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=8,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# run.finish()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.997100,0.930628
2,0.592200,0.509851
3,0.362100,0.531708
4,0.358000,0.469479
5,0.314900,0.524512
6,0.257900,0.588529
7,0.169800,0.627979
8,0.117000,0.613920


In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=6,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['transformer.layer.3.ffn.lin2.weight', 'transformer.layer.3.sa_layer_norm.bias', 'transformer.layer.6.attention.q_lin.weight', 'transformer.layer.3.sa_layer_norm.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.3.ffn.lin1.weight', 'transformer.layer.7.attention.v_lin.weight', 'transformer.layer.2.attention.v_lin.weight', 'transformer.layer.8.attention.v_lin.weight', 'transformer.layer.6.attention.k_lin.weight', 'transformer.layer.4.ffn.lin1.bias', 'transformer.layer.6.ffn.lin1.bias', 'transformer.layer.11.ffn.lin1.weight', 'transformer.layer.1.attention.k_lin.bias', 'transformer.layer.10.attention.q_lin.weight', 'transformer.layer.4.sa_layer_norm.weight', 'transformer.layer.7.sa_layer_norm.bias', 'transformer.layer.7.attention.out_lin.weight', 'transformer.layer.4.sa_layer_norm.bias', 'transformer.layer.

Epoch,Training Loss,Validation Loss
1,1.053900,1.032847
2,1.025200,0.973244
3,0.965300,0.959686
4,0.954000,0.950568
5,0.886700,0.963149
6,0.913900,0.976599


In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=15,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['transformer.layer.2.attention.q_lin.bias', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.5.sa_layer_norm.weight', 'transformer.layer.3.ffn.lin2.weight', 'transformer.layer.6.attention.out_lin.weight', 'transformer.layer.1.ffn.lin1.weight', 'transformer.layer.3.ffn.lin1.bias', 'transformer.layer.11.ffn.lin2.weight', 'transformer.layer.4.attention.k_lin.weight', 'transformer.layer.7.attention.v_lin.weight', 'transformer.layer.5.sa_layer_norm.bias', 'transformer.layer.4.output_layer_norm.weight', 'transformer.layer.4.attention.out_lin.weight', 'transformer.layer.10.ffn.lin2.bias', 'transformer.layer.9.attention.out_lin.bias', 'transformer.layer.4.attention.q_lin.weight', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.6.attention.k_lin.weight', 'transformer.layer.8.output_layer_norm.weight', 'transformer.l

Epoch,Training Loss,Validation Loss
1,1.057300,1.026046
2,1.034800,0.983806
3,0.999700,0.958144
4,0.943200,0.968173
5,0.875300,0.992532
6,0.907300,0.955654
7,0.742500,1.042039
8,0.661600,1.010859
9,0.499800,0.986476
10,0.437400,1.019951


In [ ]:
preds

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 1],
       [2, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [ ]:
preds.shape

(159, 452)

In [ ]:
np.shape(y_test)

(159, 452)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in X_raw_test for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(71868,)
(71868,)
      index        texts
0         0  <S>Pregnant
1         1  persons</S>
2         2         with
3         3  gestational
4         4     diabetes
...     ...          ...
6462     29           in
6463     30          the
6464     31      absence
6465     32           of
6466     33   screening.

[6467 rows x 2 columns]
      index        texts
0         0  <S>Pregnant
1         1  persons</S>
2         2         with
3         3  gestational
4         4     diabetes
...     ...          ...
6462     29           in
6463     30          the
6464     31      absence
6465     32           of
6466     33   screening.

[6467 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'C', 1: 'O', 2: 'E'}


In [ ]:
targetname = ['C', 'O', 'E']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)
#no determiners


       preds  gold_data
0          1       -100
1          1          1
2          1          1
3          1          1
4          1       -100
...      ...        ...
71863      1       -100
71864      1       -100
71865      1       -100
71866      1       -100
71867      1       -100

[71868 rows x 2 columns]
       preds  gold_data
1          1          1
2          1          1
3          1          1
7          1          1
12         1          1
...      ...        ...
71439      2          2
71440      2          2
71441      2          2
71442      2          2
71443      1          1

[5840 rows x 2 columns]


In [ ]:
#BioBERT - 3 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.14      0.64      0.23      1159
           E       0.25      0.43      0.31      2009
           O       0.75      0.17      0.27      7133

    accuracy                           0.27     10301
   macro avg       0.38      0.41      0.27     10301
weighted avg       0.58      0.27      0.28     10301



In [ ]:
#BioBERT - 6 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.33      0.17      0.22      1812
           C       0.14      0.41      0.21      1072
           O       0.73      0.61      0.66      6489

    accuracy                           0.50      9373
   macro avg       0.40      0.39      0.36      9373
weighted avg       0.58      0.50      0.52      9373



In [ ]:
#BioBERT - 15 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.14      0.53      0.22      1159
           E       0.22      0.42      0.29      2009
           O       0.70      0.20      0.31      7133

    accuracy                           0.28     10301
   macro avg       0.35      0.38      0.27     10301
weighted avg       0.55      0.28      0.30     10301



In [ ]:
#BioBERT - 8 epochs - zero shot
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.20      0.29      0.24      2009
           C       0.71      0.22      0.33      7133
           O       0.11      0.50      0.18      1159

    accuracy                           0.26     10301
   macro avg       0.34      0.34      0.25     10301
weighted avg       0.55      0.26      0.30     10301



In [ ]:
#BioBERT - 5 epochs - zero shot - only cause/effect labels
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.59      0.51      0.54      3299
           E       0.42      0.41      0.41      2009
           C       0.24      0.34      0.28      1159

    accuracy                           0.45      6467
   macro avg       0.41      0.42      0.41      6467
weighted avg       0.47      0.45      0.46      6467



In [ ]:
#BioBERT - 8 epochs - zero shot - only cause/effect labels
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.65      0.35      0.45      3299
           E       0.40      0.37      0.38      2009
           C       0.22      0.53      0.31      1159

    accuracy                           0.39      6467
   macro avg       0.42      0.42      0.38      6467
weighted avg       0.49      0.39      0.41      6467



In [ ]:
#BioBERT - 8 epochs - zero shot - only cause/effect labels - final
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.62      0.68      0.65      2009
           O       0.74      0.51      0.61      3299
           C       0.41      0.70      0.52      1159

    accuracy                           0.60      6467
   macro avg       0.59      0.63      0.59      6467
weighted avg       0.64      0.60      0.60      6467



In [ ]:
#BioBERT - 8 epochs - few shot
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.73      0.77      0.75      6489
           C       0.21      0.11      0.14      1072
           E       0.31      0.33      0.32      1812

    accuracy                           0.61      9373
   macro avg       0.42      0.41      0.41      9373
weighted avg       0.59      0.61      0.60      9373



In [ ]:
#BioBERT - 8 epochs - few shot - only cause/effect labels
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.65      0.62      0.64      3044
           E       0.45      0.40      0.42      1795
           C       0.27      0.34      0.30      1001

    accuracy                           0.51      5840
   macro avg       0.45      0.46      0.45      5840
weighted avg       0.52      0.51      0.51      5840



In [ ]:
#BioBERT - 8 epochs - few shot - only cause/effect labels - final
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.59      0.63      0.61      1001
           O       0.80      0.76      0.78      3044
           E       0.71      0.75      0.73      1795

    accuracy                           0.73      5840
   macro avg       0.70      0.71      0.71      5840
weighted avg       0.74      0.73      0.74      5840



In [ ]:
#############randomly add 10% of medical data to the organizational data during fine-tuning#############################################

In [ ]:
#BioBERT - 15 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.17      0.15      0.16      1072
           O       0.73      0.74      0.74      6489
           E       0.31      0.32      0.32      1812

    accuracy                           0.59      9373
   macro avg       0.40      0.40      0.40      9373
weighted avg       0.59      0.59      0.59      9373



In [ ]:
#############randomly add 10% of medical data to the organizational data during fine-tuning#############################################

In [ ]:
preds.shape

(260, 417)

In [ ]:
np.shape(y_test)


(260, 417)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in X_raw_test for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(108420,)
(108420,)
       index        texts
0          0     Pregnant
1          1      persons
2          2         with
3          3  gestational
4          4     diabetes
...      ...          ...
10296     29           in
10297     30          the
10298     31      absence
10299     32           of
10300     33   screening.

[10301 rows x 2 columns]
       index        texts
0          0     Pregnant
1          1      persons
2          2         with
3          3  gestational
4          4     diabetes
...      ...          ...
10296     29           in
10297     30          the
10298     31      absence
10299     32           of
10300     33   screening.

[10301 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'C', 1: 'O', 2: 'E'}


In [ ]:
targetname = ['C', 'O', 'E']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)
#no determiners


        preds  gold_data
0           1       -100
1           0          1
2           0          1
3           1          1
4           0          1
...       ...        ...
108415      1       -100
108416      1       -100
108417      1       -100
108418      1       -100
108419      1       -100

[108420 rows x 2 columns]
        preds  gold_data
1           0          1
2           0          1
3           1          1
4           0          1
7           0          1
...       ...        ...
108045      1          1
108046      1          1
108047      1          1
108048      1          1
108049      1          1

[9373 rows x 2 columns]


In [ ]:
#distilbert base uncased
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.44      0.34      0.38      1072
           O       0.83      0.87      0.85      6489
           E       0.67      0.65      0.66      1812

    accuracy                           0.77      9373
   macro avg       0.65      0.62      0.63      9373
weighted avg       0.75      0.77      0.76      9373



In [ ]:
############BERT

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification

In [ ]:
model_name = "bert-base-uncased" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
#zero-shot
import wandb

run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BertforTokenClassification',
    num_train_epochs=100,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
run.finish()

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['transformer.layer.6.attention.q_lin.bias', 'transformer.layer.7.ffn.lin1.bias', 'transformer.layer.1.ffn.lin1.weight', 'transformer.layer.2.ffn.lin1.weight', 'transformer.layer.5.attention.out_lin.bias', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.1.ffn.lin2.bias', 'transformer.layer.8.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.10.sa_layer_norm.weight', 'transformer.layer.11.ffn.lin1.bias', 'transformer.layer.4.attention.k_lin.weight', 'transformer.layer.2.attention.k_lin.bias', 'transformer.layer.11.output_layer_norm.bias', 'transformer.layer.3.attention.out_lin.weight', 'transformer.layer.8.ffn.lin2.weight', 'transformer.layer.4.ffn.lin2.weight', 'transformer.layer.3.output_layer_norm.bias', 'transformer.layer.2.output_layer_norm.weight', 't

Epoch,Training Loss,Validation Loss
1,1.052200,1.010367
2,1.005300,0.975310
3,0.961100,0.950147
4,0.938600,1.007233
5,0.896700,0.978880
6,0.863200,1.005233
7,0.760000,0.949408
8,0.641700,0.967138
9,0.618300,1.034922
10,0.447100,1.075684


In [ ]:
#zero-shot
# import wandb

# run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BertforTokenClassification',
    num_train_epochs=7,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# run.finish()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.008700,0.970698
2,0.687400,0.625827
3,0.458400,0.472579
4,0.325500,0.467605
5,0.286600,0.514187
6,0.228800,0.517472
7,0.251100,0.549494


In [ ]:
#Few-shot
# import wandb

# run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BertforTokenClassification',
    num_train_epochs=7,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# run.finish()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.007600,0.962561
2,0.710600,0.604463
3,0.412400,0.543343
4,0.390200,0.486759
5,0.300500,0.515550
6,0.240900,0.641602
7,0.172400,0.642529


In [ ]:
#zero-shot
# import wandb

# run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# run.finish()

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['transformer.layer.6.sa_layer_norm.bias', 'transformer.layer.10.attention.v_lin.bias', 'embeddings.position_embeddings.weight', 'embeddings.LayerNorm.weight', 'transformer.layer.8.attention.k_lin.weight', 'transformer.layer.3.attention.k_lin.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.11.sa_layer_norm.weight', 'transformer.layer.1.attention.out_lin.bias', 'transformer.layer.11.attention.out_lin.weight', 'transformer.layer.6.ffn.lin1.bias', 'transformer.layer.11.attention.out_lin.bias', 'transformer.layer.11.output_layer_norm.bias', 'transformer.layer.4.sa_layer_norm.weight', 'transformer.layer.9.ffn.lin1.weight', 'transformer.layer.8.output_layer_norm.weight', '

Epoch,Training Loss,Validation Loss
1,1.041000,1.002963
2,1.015300,0.974619
3,0.961200,0.948939


In [ ]:
preds.shape

(177, 424)

In [ ]:
np.shape(y_test)

(177, 424)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in X_raw_test for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(75048,)
(75048,)
      index        texts
0         0  <S>Pregnant
1         1  persons</S>
2         2         with
3         3  gestational
4         4     diabetes
...     ...          ...
6462     29           in
6463     30          the
6464     31      absence
6465     32           of
6466     33   screening.

[6467 rows x 2 columns]
      index        texts
0         0  <S>Pregnant
1         1  persons</S>
2         2         with
3         3  gestational
4         4     diabetes
...     ...          ...
6462     29           in
6463     30          the
6464     31      absence
6465     32           of
6466     33   screening.

[6467 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'E', 1: 'O', 2: 'C'}


In [ ]:
targetname = ['E', 'O', 'C']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)
#no determiners


       preds  gold_data
0          2       -100
1          1          1
2          1       -100
3          1       -100
4          2       -100
...      ...        ...
75043      1       -100
75044      1       -100
75045      0       -100
75046      0       -100
75047      0       -100

[75048 rows x 2 columns]
       preds  gold_data
1          1          1
5          2          1
10         2          2
11         2          2
14         2          2
...      ...        ...
74661      0          1
74662      0          1
74663      0          1
74664      0          1
74665      0          1

[6467 rows x 2 columns]


In [ ]:
#BERT - 3 epochs - zero shot
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.77      0.14      0.24      7133
           C       0.13      0.68      0.22      1159
           E       0.23      0.35      0.28      2009

    accuracy                           0.24     10301
   macro avg       0.38      0.39      0.25     10301
weighted avg       0.59      0.24      0.24     10301



In [ ]:
#BERT - 7 epochs - zero shot
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.69      0.18      0.29      7133
           C       0.14      0.68      0.23      1159
           E       0.23      0.33      0.27      2009

    accuracy                           0.26     10301
   macro avg       0.35      0.39      0.26     10301
weighted avg       0.54      0.26      0.28     10301



In [ ]:
#BERT - 7 epochs - zero shot - only cause/effect
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.70      0.32      0.44      3299
           E       0.52      0.56      0.54      2009
           C       0.30      0.73      0.43      1159

    accuracy                           0.47      6467
   macro avg       0.51      0.54      0.47      6467
weighted avg       0.57      0.47      0.47      6467



In [ ]:
#BERT - 7 epochs - zero shot - only cause/effect - final
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.47      0.54      0.51      2009
           O       0.70      0.39      0.50      3299
           C       0.31      0.63      0.42      1159

    accuracy                           0.48      6467
   macro avg       0.50      0.52      0.48      6467
weighted avg       0.56      0.48      0.49      6467



In [ ]:
#BERT - 7 epochs - Few shot randomly adding 10% data
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.71      0.85      0.77      6489
           C       0.16      0.10      0.12      1072
           E       0.32      0.17      0.22      1812

    accuracy                           0.63      9373
   macro avg       0.39      0.37      0.37      9373
weighted avg       0.57      0.63      0.59      9373



In [ ]:
#BERT - 7 epochs - Few shot randomly adding 10% data - cause/effect only
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.83      0.55      0.66      3044
           E       0.65      0.58      0.61      1795
           C       0.35      0.78      0.48      1001

    accuracy                           0.60      5840
   macro avg       0.61      0.64      0.59      5840
weighted avg       0.69      0.60      0.62      5840

